In [ ]:
#Stick to a notebook for now until everything everything has been structured and works. 

In [5]:
# Rewrite the data to a csv file first.
import csv

all_lines = []
with open("/home/frederik/auterion/stl_gen/swissSURFACE3D_Raster_example/swissSURFACE3D_Raster_0.5_xyz_CHLV95_LN02_2695_1259.xyz") as orig_file:
    all_lines = orig_file.readlines()
    orig_file.close()

csv_file_path = "/home/frederik/auterion/stl_gen/processed_data/data.csv"

# Open the CSV file for writing
with open(csv_file_path, "w+", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)

    # Write the data to the CSV file
    for row in all_lines:
        csv_writer.writerow(row.split())

In [2]:
# With the csv file we can now go through and generate triangles
import csv
import numpy as np

csv_file_path = "/home/frederik/auterion/stl_gen/processed_data/data.csv"
data_list = []
with open(csv_file_path,'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    _ = next(csv_reader)

    for row in csv_reader:
        data_list.append(row)

    csv_file.close()

data = np.array(data_list)

data = data.astype(float)

print(data.shape)

# Find the dataset corners in 2D coordinates
max_x = max(data[:,0])
min_x = min(data[:,0])
max_y = max(data[:,1])
min_y = min(data[:,1])

(4000000, 3)


In [11]:
# For every point you generate two separate triangles.
# Checks that need to be done:
# if y == max_y, no restrictions
# if y == min_y, no triangle calculation
# if x == min_x, no second triangle calculations
# if x == max_x, no first triangle calculations

# Coordinate System and triangle orientation
# -------> y
# |
# |                 Current -----> Front
# |               ↗    |↑           /
# V             /      ||   1.    /
#             /        ||       /
#           /    2.    ||     /
#         /            V|   /
#     Diagonal<------Below ↙   
#
from decimal import Decimal

def write_block(file,vertex1,vertex2,vertex3):
    vec1 = vertex2 - vertex1
    vec2 = vertex3 - vertex1

    normal = np.zeros(shape=(3))

    normal[0] = round(vec1[1]*vec2[2] - vec2[1]*vec1[2],3)
    normal[1] = round(vec1[2]*vec2[0] - vec2[2]*vec1[0],3)
    normal[2] = round(vec1[0]*vec2[1] - vec2[0]*vec1[1],3)

    with open(file,'a') as stl_file:
        stl_file.write(f'facet normal {"{:e}".format(normal[0])} {"{:e}".format(normal[1])} {"{:e}".format(normal[2])} \n')
        stl_file.write("\touter loop\n")
        stl_file.write(f'\t\tvertex {"{:e}".format(vertex1[0])} {"{:e}".format(vertex1[1])} {"{:e}".format(vertex1[2])}\n')
        stl_file.write(f'\t\tvertex {"{:e}".format(vertex2[0])} {"{:e}".format(vertex2[1])} {"{:e}".format(vertex2[2])}\n')
        stl_file.write(f'\t\tvertex {"{:e}".format(vertex3[0])} {"{:e}".format(vertex3[1])} {"{:e}".format(vertex3[2])}\n')
        stl_file.write("\tendloop\n")
        stl_file.write("endfacet\n\n")
        stl_file.close()


STL_FILE = "/home/frederik/auterion/stl_gen/processed_data/stl_file.stl"

# Write the header line into the stl file
with open(STL_FILE,'w') as stl_file:
    stl_file.write("solid\n")
    stl_file.close()

#This algorithm only works for raster structures. Irregular point distributions will not be able to use this.
for i, xyz in enumerate(data):

    # Skip calculation as no triangles can be generated.
    if(xyz[1] == min_y):
        continue

    current = xyz
    front = None
    below = None
    diagonal = None
    # Calculate first triangle
    if xyz[0] != max_x:
        front = data[i+1]
        below = data[i+2000] # Tile is 1km x 1km with 50 cm spacing between points. The point 
        write_block(STL_FILE,current,front,below)
        # front = data[]

    # Calculate second triangle
    if xyz[0] != min_x:
        below = data[i+2000]
        diagonal = data[i+1999]
        write_block(STL_FILE,current,below,diagonal)
    

with open(STL_FILE,'a') as stl_file:
    stl_file.write("endsolid")
    stl_file.close()


1.234000e-06
